# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,la passe,45.5549,-0.8967,289.47,68,67,2.64,FR,1728639165
1,1,quibdo,5.6947,-76.6611,294.84,99,90,0.80,CO,1728639166
2,2,iqaluit,63.7506,-68.5145,274.00,80,100,3.09,CA,1728638958
3,3,puerto natales,-51.7236,-72.4875,274.89,92,83,2.72,CL,1728639168
4,4,alice springs,-23.7000,133.8833,303.90,11,100,4.63,AU,1728639169


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    alpha = 0.6,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
criteria_city_data_df = pd.DataFrame(city_data_df.loc[((city_data_df['Max Temp'] - 273.15) > 21) & 
    ((city_data_df['Max Temp'] - 273.15) < 27) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0),:])

# Drop any rows with null values
criteria_city_data_df = criteria_city_data_df.dropna()

# Convert from Kelvin to Celsius
criteria_city_data_df['Max Temp']= criteria_city_data_df['Max Temp'] - 273.15

# Display sample data
criteria_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,hilo,19.7297,-155.0900,22.28,98,0,2.57,US,1728639179
50,50,remire-montjoly,4.9167,-52.2667,24.02,94,0,0.51,GF,1728639222
53,53,san antonio,29.4241,-98.4936,21.88,69,0,1.54,US,1728639002
96,96,crane,31.3974,-102.3501,21.16,38,0,4.33,US,1728639278
122,122,ciudad constitucion,25.0369,-111.6644,23.08,70,0,1.38,MX,1728639307
141,141,ma'ili,21.4219,-158.1803,26.01,78,0,1.54,US,1728639331
250,250,qa'en,33.7265,59.1844,26.20,10,0,0.72,IR,1728639459
297,297,hawaiian paradise park,19.5933,-154.9731,22.16,99,0,2.57,US,1728639516
366,366,kashgar,39.4547,75.9797,24.56,23,0,2.00,CN,1728639603
428,428,baramula,34.2000,74.3500,25.58,17,0,1.92,IN,1728639682


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = criteria_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
13,hilo,US,19.7297,-155.0900,98,
50,remire-montjoly,GF,4.9167,-52.2667,94,
53,san antonio,US,29.4241,-98.4936,69,
96,crane,US,31.3974,-102.3501,38,
122,ciudad constitucion,MX,25.0369,-111.6644,70,
141,ma'ili,US,21.4219,-158.1803,78,
250,qa'en,IR,33.7265,59.1844,10,
297,hawaiian paradise park,US,19.5933,-154.9731,99,
366,kashgar,CN,39.4547,75.9797,23,
428,baramula,IN,34.2000,74.3500,17,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",  # Search for hotels
    "filter": "",  # Set the location and radius
    "bias": "",  # Bias towards the city's location
    "limit": 1,  # Get only the first result
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hilo - nearest hotel: Dolphin Bay Hotel
remire-montjoly - nearest hotel: Complexe Belova
san antonio - nearest hotel: Riverwalk Plaza Hotel and Suites
crane - nearest hotel: No hotel found
ciudad constitucion - nearest hotel: Hotel El Arbolito
ma'ili - nearest hotel: No hotel found
qa'en - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
kashgar - nearest hotel: 深航国际酒店
baramula - nearest hotel: No hotel found
mvurwi - nearest hotel: No hotel found
zabol - nearest hotel: No hotel found
aral - nearest hotel: No hotel found
mutare - nearest hotel: Amber Sun hotel
piti village - nearest hotel: The Cliff Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
13,hilo,US,19.7297,-155.0900,98,Dolphin Bay Hotel
50,remire-montjoly,GF,4.9167,-52.2667,94,Complexe Belova
53,san antonio,US,29.4241,-98.4936,69,Riverwalk Plaza Hotel and Suites
96,crane,US,31.3974,-102.3501,38,No hotel found
122,ciudad constitucion,MX,25.0369,-111.6644,70,Hotel El Arbolito
141,ma'ili,US,21.4219,-158.1803,78,No hotel found
250,qa'en,IR,33.7265,59.1844,10,No hotel found
297,hawaiian paradise park,US,19.5933,-154.9731,99,No hotel found
366,kashgar,CN,39.4547,75.9797,23,深航国际酒店
428,baramula,IN,34.2000,74.3500,17,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.7,
    color = "City",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)